## BERT Knowledge Representation
We want to compute, at each step of the user session (i.e. each document clicked), how their internal knowledge representation changes. Therefore, we have a few different methods to do so using Bert embeddings as a starting point.
A few assumptions:
- user starts with an empty knowledge representation
- User READS every document, and that is added to their knowledge

Document embeddings for BERT can come in a few different forms. Check `Compute_BERT_embeddings.ipynb` for how we compute each:
- SUM: sum of the embeddings for each sentence of the document
- MEAN: mean of the embeddings for each sentence of the document
- TRUNC: Truncate the document at the first 384 tokens.
- maxp_pairwise: Considering all sentences from the Wikipedia topic and the document, consider only the sentence with the higher similarity for any Wikipedia sentence
- maxp_sum: Consider only the sentence with higher similarity to the SUM of the wikipedia sentences
- maxp_mean: Consider only the sentence with higher similarity to the MEAN of the wikipedia sentences
- maxp_trunc: Consider only the sentence with higher similarity to the truncated wikipedia document
    
These are the ways we can compute the users' knowledge evolution. Will be compared to the same method of aggregation on the Wikipedia text

- MEAN: Concatenate all of the documents, the MEAN of these is the final knowledge.
- SUM: As the user clicks on documents, SUM the embeddings


In [64]:
import json
import pickle
import urllib.parse

from collections import OrderedDict, defaultdict
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np

from scipy.spatial.distance import cosine, euclidean
from scipy.stats import kendalltau, pearsonr, spearmanr
from sklearn.preprocessing import normalize
from tqdm.auto import tqdm

In [ ]:
normalized = True
base_model, hidden_size = "msmarco-MiniLM-L6-cos-v5", 384

f_dist = cosine


def load_embeddings(name, doc="docs", normalized=normalized):
    if normalized:
        if doc == "wikipedia":
            return pickle.load(open(f"../data/bert_embeddings/{base_model}_{doc}_{name}_embeddings_norm.pkl", "rb"))
        return pickle.load(open(f"../data/bert_embeddings/{base_model}_{doc}_{name}_embeddings_norm.pkl", "rb"))

    if doc == "wikipedia":
        return pickle.load(open(f"../data/bert_embeddings/{base_model}_{doc}_{name}_embeddings.pkl", "rb"))
    return pickle.load(open(f"../data/bert_embeddings/{base_model}_{doc}_{name}_embeddings.pkl", "rb"))


dataset = json.load(open("../data/logs_with_position.json"))

embeddings = {
    "docs_mean": load_embeddings("mean", normalized=False),
    "docs_sum": load_embeddings("sum", normalized=False),
    "docs_trunc": load_embeddings("trunc", normalized=False),
}

wikipedia_embeddings = {
    "sum": load_embeddings("sum", "wikipedia", normalized=False),
    "mean": load_embeddings("mean", "wikipedia", normalized=False),
    "trunc": load_embeddings("trunc", "wikipedia", normalized=False),
}

embeddings_norm = {
    "docs_mean": load_embeddings("mean", normalized=True),
    "docs_sum": load_embeddings("sum", normalized=True),
    "docs_trunc": load_embeddings("trunc", normalized=True),
}

wikipedia_embeddings_norm = {
    "sum": load_embeddings("sum", "wikipedia", normalized=True),
    "mean": load_embeddings("mean", "wikipedia", normalized=True),
    "trunc": load_embeddings("trunc", "wikipedia", normalized=True),
}

In [81]:
# sum all around only.

missing_docs = set()

embs = embeddings["docs_sum"]
wiki_embs = wikipedia_embeddings["sum"]
final_knowledges = OrderedDict()

for u in tqdm(dataset):
    u_id = u["userID"]
    ALG = u["ALG"]
    RPL = u["RPL"]
    user_knowledge = np.zeros(hidden_size)  # initialize knowledge as zeros.
    topic = urllib.parse.quote(u["topic_title"])

    for d in u["clicks"]:
        url = d["url"]
        if url not in embs:
            missing_docs.add(url)
            continue
        user_knowledge += embs[url]

    final_knowledges[u_id] = {"RPL": RPL, "ALG": ALG, "estimated": 1 - f_dist(user_knowledge, wiki_embs[topic])}

  0%|          | 0/127 [00:00<?, ?it/s]

## Correlation between learning and embedding methods

In [82]:
u_ids = final_knowledges.keys()
RPLs = [final_knowledges[u]["RPL"] for u in u_ids]
results = [final_knowledges[u]["estimated"] for u in u_ids]
pearsons = pearsonr(results, RPLs)
print(pearsons)
print(f"missing docs: {len(missing_docs)}")

pickle.dump(dict(final_knowledges), open(f"../data/{base_model}_knowledge_gains.pkl", "wb"))

(0.2923127508093131, 0.0008536912371208248)
missing docs: 33


# Old

In [30]:
methods = list(embeddings.keys())
users_knowledge_MEAN = []  # add final score for the user
users_knowledge_SUM = []
final_knowledges = defaultdict(OrderedDict)
missing_docs = set()

for u in dataset:
    u_id = u["userID"]
    ALG = u["ALG"]
    RPL = u["RPL"]

    user_knowledge_mean = {k: [] for k in methods}
    user_knowledge_sum = {k: np.zeros(hidden_size) for k in methods}
    topic = urllib.parse.quote(u["topic_title"])
    for d in u["clicks"]:
        url = d["url"]
        if url not in embeddings["docs_mean"] or not np.any(embeddings["docs_mean"][url]):
            missing_docs.add(url)
            continue
        for method in methods:
            emb = embeddings[method][url]
            user_knowledge_mean[method].append(emb)
            user_knowledge_sum[method] += emb
    # normalize and compute final similarity
    for method in methods:
        if normalized:
            knowledge_mean = normalize(np.mean(user_knowledge_mean[method], axis=0).reshape(1, -1)).flatten()
            knowledge_sum = normalize(user_knowledge_sum[method].reshape(1, -1)).flatten()
        else:
            knowledge_mean = np.mean(user_knowledge_mean[method], axis=0)
            knowledge_sum = user_knowledge_sum[method]

        for emb_type in wikipedia_embeddings.keys():
            wiki_emb = wikipedia_embeddings[emb_type][topic]
            sum_dict = {"RPL": RPL, "ALG": ALG, "final_sim": 1 - f_dist(knowledge_sum, wiki_emb)}
            mean_dict = {"RPL": RPL, "ALG": ALG, "final_sim": 1 - f_dist(knowledge_mean, wiki_emb)}

            final_knowledges[f"SUM_{method}_wiki_{emb_type}"][u_id] = sum_dict
            final_knowledges[f"MEAN_{method}_wiki_{emb_type}"][u_id] = mean_dict

pickle.dump(dict(final_knowledges), open(f"../data/{base_model}_knowledge_gains.pkl", "wb"))
with open("../data/missing_docs.txt", "w") as outf:
    for u in missing_docs:
        outf.write(f"{u}\n")

## Correlation between learning and embedding methods

In the end, we want to know, for each of the 42 methods tried above, which one has the higher correlation with learning gains.

In [32]:
RPLs = []
ALGs = []
u_ids = [u["userID"] for u in dataset]

# make sure we follow the same order of users
pearsons_ALG = {}
spearman_ALG = {}
kendalltau_ALG = {}
corr_ALG = {}

for m in final_knowledges:
    u_ids = final_knowledges[m].keys()
    ALGs = [final_knowledges[m][u]["ALG"] for u in u_ids]
    RPLs = [final_knowledges[m][u]["RPL"] for u in u_ids]
    results = [final_knowledges[m][u]["final_sim"] for u in u_ids]

    # results = [_users[x]["final_sim"] for x in u_ids if x in _users]
    pearsons_ALG[m] = pearsonr(results, RPLs)
    spearman_ALG[m] = spearmanr(results, RPLs)

    # pearsons_ALG[m] = pearsonr(results, ALGs)
    # spearman_ALG[m] = spearmanr(results, ALGs)


print(base_model)
pprint(sorted(pearsons_ALG.items(), key=lambda x: x[1], reverse=True)[:3])
pprint(sorted(spearman_ALG.items(), key=lambda x: x[1], reverse=True)[:3])

msmarco-MiniLM-L6-cos-v5
[('MEAN_docs_sum_wiki_sum', (0.26409135116625926, 0.0026981601798359072)),
 ('MEAN_docs_sum_wiki_mean', (0.2640913339130606, 0.002698161980537756)),
 ('MEAN_docs_mean_wiki_mean', (0.26409131699651944, 0.0026981637461039798))]
[('SUM_docs_mean_wiki_sum',
  SpearmanrResult(correlation=0.31397986988277615, pvalue=0.0003247214445397043)),
 ('MEAN_docs_mean_wiki_sum',
  SpearmanrResult(correlation=0.31397986988277615, pvalue=0.0003247214445397043)),
 ('SUM_docs_mean_wiki_mean',
  SpearmanrResult(correlation=0.31397986988277615, pvalue=0.0003247214445397043))]
